In [3]:
from langgraph.graph import Graph
from langchain_groq import ChatGroq


In [4]:
llm = ChatGroq(model="llama3-70b-8192")

llm.invoke("HI how are you")

AIMessage(content="I'm doing well, thanks! I'm a large language model, so I don't have feelings like humans do, but I'm always happy to chat with you and help with any questions or topics you'd like to discuss. How about you? How's your day going so far?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 14, 'total_tokens': 73, 'completion_time': 0.182026055, 'prompt_time': 0.003424535, 'queue_time': 0.017563474, 'total_time': 0.18545059}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_753a4aecf6', 'finish_reason': 'stop', 'logprobs': None}, id='run-c3fd0f9c-6356-44bd-96a9-b8db2c0a5071-0', usage_metadata={'input_tokens': 14, 'output_tokens': 59, 'total_tokens': 73})

In [8]:
from langgraph.graph import Graph, START, END
from langchain_groq import ChatGroq
from typing import TypedDict, Annotated, Sequence, Dict, Any
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage
import json
from operator import itemgetter

# Define our state
class AgentState(TypedDict):
    messages: Sequence[BaseMessage]
    current_step: str
    customer_issue: str
    proposed_solution: str

# Initialize the LLM
llm = ChatGroq(
    model="llama3-70b-8192",
    temperature=0.7,
)

# Add a callback function to display current step
def display_step(state: AgentState) -> AgentState:
    """Display the current step of the workflow"""
    step_emojis = {
        "greet": "👋",
        "understand_issue": "🤔",
        "provide_solution": "💡",
        "end": "✅"
    }
    current_step = state["current_step"]
    emoji = step_emojis.get(current_step, "")
    print(f"\n[Current Step: {emoji} {current_step}]\n")
    return state

#===================== NODES ===================
def greet_customer(state: AgentState) -> AgentState:
    display_step(state)
    messages = state["messages"]
    
    system_prompt = SystemMessage(content="""You are a helpful customer support agent. 
    Greet the customer warmly and ask about their issue.""")
    
    response = llm.invoke([system_prompt] + list(messages))
    
    return {
        **state,
        "messages": [*messages, response],
        "current_step": "understand_issue"
    }

def understand_issue(state: AgentState) -> AgentState:
    display_step(state)
    messages = state["messages"]
    
    system_prompt = SystemMessage(content="""Analyze the customer's issue and summarize it clearly. 
    Ask any clarifying questions if needed.""")
    
    response = llm.invoke([system_prompt] + list(messages))
    
    return {
        **state,
        "messages": [*messages, response],
        "current_step": "provide_solution",
        "customer_issue": response.content
    }

def provide_solution(state: AgentState) -> AgentState:
    display_step(state)
    messages = state["messages"]
    
    system_prompt = SystemMessage(content="""Based on the customer's issue, provide a clear and helpful solution. 
    Be specific and ask if they need any clarification.""")
    
    response = llm.invoke([system_prompt] + list(messages))
    
    return {
        **state,
        "messages": [*messages, response],
        "current_step": "end",
        "proposed_solution": response.content
    }

# 
def should_continue(state: AgentState) -> str:
    """Determine if we should continue the conversation"""
    if state["current_step"] == "end":
        return "end"
    return state["current_step"]

# Create the graph
workflow = Graph()

# Add nodes
workflow.add_node("greet_cutomer", greet_customer)
workflow.add_node("understand_issue", understand_issue)
workflow.add_node("provide_solution", provide_solution)

# Add edges
workflow.add_edge(START, "greet_cutomer")
workflow.add_edge("greet_cutomer", "understand_issue")
workflow.add_edge("understand_issue", "provide_solution")
workflow.add_edge("provide_solution", END)

# Compile the graph
chain = workflow.compile()

# Function to run the agent
def run_customer_support(customer_message: str):
    """Run the customer support workflow"""
    initial_state = {
        "messages": [HumanMessage(content=customer_message)],
        "current_step": "greet",
        "customer_issue": "",
        "proposed_solution": ""
    }
    
    result = chain.invoke(initial_state)
    
    # Return all messages for display
    return [msg.content for msg in result["messages"]]

# Test the agent
customer_query = "Hi, I'm having trouble logging into my account. It keeps saying invalid password even though I'm sure it's correct."
conversation = run_customer_support(customer_query)

# Print the conversation
for message in conversation:
    print(message + "\n")


[Current Step: 👋 greet]


[Current Step: 🤔 understand_issue]


[Current Step: 💡 provide_solution]

Hi, I'm having trouble logging into my account. It keeps saying invalid password even though I'm sure it's correct.

I'm so sorry to hear that you're having trouble logging into your account! Thank you for reaching out to us for help.

Can you please tell me a little bit more about the issue you're experiencing? For example, have you tried resetting your password or logging in from a different device or browser? Additionally, do you remember the last time you were able to log in successfully?

Also, just to confirm, you're certain that you're entering the correct username and password combination, and that your caps lock key isn't accidentally enabled, right?



Let's try a few troubleshooting steps to help you regain access to your account.

First, let's try resetting your password. You can do this by clicking on the "Forgot Password" link on the login page. Enter your username or email